This notebook contains data prep for Tableau maps

In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
import numpy as np

In [5]:
# Loading in data 
# Import Census broadband availability data (accessed and extracted from ) 
bb = pd.read_csv('/Users/tm/LinkNYC/Broadband/data/policymap_broadband_nyc.csv', usecols=['Census Tract','% of households w/o broadband'])
snap = pd.read_csv('/Users/tm/LinkNYC/Broadband/data/policymap_est_percent_of_all_families_rec_SNAP_benefits.csv')


# shapefile can be downloaded here: https://data.cityofnewyork.us/api/geospatial/fxpq-c8ku?method=export&format=Shapefile
# Import the shapefile 
ctshp = gpd.GeoDataFrame.from_file('/Users/tm/LinkNYC/Broadband/data/2010 Census Tracts/geo_export_e0b88341-f65f-4021-8eed-96940660bc89.shp')
#ctshp.head(3)

In [6]:
bb = pd.merge(bb, snap, on='Census Tract')

In [10]:
bb.head(10)

,Census Tract,% of households w/o broadband,FIPS Code,Formatted FIPS,% on SNAP
0,"36081990100, NY",NaN,3.608199e+10,3.608199e+10,NaN
1,"36085990100, NY",NaN,3.608599e+10,3.608599e+10,NaN
2,"36081107202, NY",NaN,3.608111e+10,3.608111e+10,NaN
3,"36047070203, NY",NaN,3.604707e+10,3.604707e+10,NaN
4,"36081071600, NY",NaN,3.608107e+10,3.608107e+10,NaN
5,"36047990100, NY",NaN,3.604799e+10,3.604799e+10,NaN
6,"36005050400, NY",27.50,3.600505e+10,3.600505e+10,72.50
7,"36085022800, NY",0.00,3.608502e+10,3.608502e+10,0.00
8,"36085029102, NY",13.98,3.608503e+10,3.608503e+10,3.53
9,"36085019800, NY",21.82,3.608502e+10,3.608502e+10,2.24


__About census tract codes:__

Census tracts are geographic entities within counties (or the statistical equivalent of counties). The entire area of a county is covered by census tracts, just as the entire area of a state is covered by counties and county equivalents.

Per FCC, "..a census tract code may not be used more than once in a single county, but it may be used again in a different county in the same state or in a county in a different state."

Census Tracts consist of 3 things: 

![Image denotes census tract number breakdown, 
with the first two digits corresponding to the state,
the following three digits corresponding to the county, 
and the last six digits corresping to the census tract code (four digits) and a suffix (two digits)](data/images/census_tract_code_breakdown.png)


- The first two digits corresponding to the state,
- The following three digits corresponding to the county, 
- The last six digits corresping to the census tract code (four digits) and a suffix (two digits)

More here: https://transition.fcc.gov/form477/Geo/more_about_census_tracts.pdf

In [11]:
# Extract the county codes and census tracts from census tract code 
# These numbers stand for the following NYC Boroughs & counties:

# 061 - Manhattan (New York County)
# 005 - The Bronx (Bronx County)
# 047 - Brooklyn (Kings County)
# 081 - Queens (Queens Count)
# 085 - Staten Island (Richmond County)


bb['census_tract_county_code'] = bb['Census Tract'].str[2:5] # digits 3,4,5
bb['census_tract'] = bb['Census Tract'].str[5:11] # last 6 digits

In [12]:
bb['borough'] = np.nan

In [13]:
# Create NYC borough codes out of county codes. 
#
# The City of New York uses the following codes for boroughs
# 1 - Manhattan
# 2 - The Bronx
# 3 - Brooklyn
# 4 - Queens
# 5 - Staten Island

conditions = [
    (bb['census_tract_county_code'] =='061'),
    (bb['census_tract_county_code'] =='005'),
    (bb['census_tract_county_code'] =='047'),
    (bb['census_tract_county_code'] =='081'),
    (bb['census_tract_county_code'] =='085'),
]

choices = [1,2,3,4,5]

bb['borough'] = np.select(conditions, choices, default='!!!')

In [14]:
# Create a column for borough + census tract 7-digit code as used by NYC Open Data 
bb['borough_census_tract'] = bb['borough'] + bb['census_tract']

In [15]:
# bb['% of households w/o broadband'].isnull().sum()

In [16]:
bb.head()

,Census Tract,% of households w/o broadband,FIPS Code,Formatted FIPS,% on SNAP,census_tract_county_code,census_tract,borough,borough_census_tract
0,"36081990100, NY",NaN,3.608199e+10,3.608199e+10,NaN,081,990100,4,4990100
1,"36085990100, NY",NaN,3.608599e+10,3.608599e+10,NaN,085,990100,5,5990100
2,"36081107202, NY",NaN,3.608111e+10,3.608111e+10,NaN,081,107202,4,4107202
3,"36047070203, NY",NaN,3.604707e+10,3.604707e+10,NaN,047,070203,3,3070203
4,"36081071600, NY",NaN,3.608107e+10,3.608107e+10,NaN,081,071600,4,4071600


In [17]:
ctshp_bb = pd.merge(ctshp, bb, left_on='boro_ct201', right_on='borough_census_tract')

In [18]:
ctshp_bb.to_file("ctshp_bb_1.shp")